In [2]:
from google.colab import drive

drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [3]:
!pip install transformers
!pip install --upgrade transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [2]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/Langchain/gm_fashions_dataset.csv'
data = pd.read_csv(file_path)

pd.set_option('display.max_rows', None)
data


,Question,Answer,Type
0,What is GM Fashions?,GM Fashions is an online clothing store offeri...,train
1,Where is GM Fashions located?,"GM Fashions is primarily located in Chennai, I...",train
2,How can I contact GM Fashions?,You can contact GM Fashions via email at suppo...,train
3,What categories of clothing does GM Fashions o...,GM Fashions offers men's clothing categories s...,train
4,What special sections are available at GM Fash...,GM Fashions has a special Offer Zone that feat...,train
5,What is the primary location of GM Fashions?,The primary location of GM Fashions is Chennai...,train
6,What are the showroom locations of GM Fashions?,GM Fashions has multiple showroom locations ac...,test
7,What is the head office address of GM Fashions?,The head office address of GM Fashions is in C...,test


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = '/content/drive/MyDrive/Colab Notebooks/Langchain/Distilgpt'

# Load the tokenizer and model using AutoTokenizer and AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

print(model.__class__.__name__)
print(tokenizer.__class__.__name__)
print(model.config)

GPT2LMHeadModel
GPT2TokenizerFast
GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/Langchain/Distilgpt",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "tra

In [4]:
input_text = "What special sections are available at GM Fash...	"
inputs = tokenizer(input_text, return_tensors='pt')
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


What special sections are available at GM Fash...	.com.








In [5]:
from datasets import Dataset

# Rename columns for clarity
data = data.rename(columns={"Question": "input_text", "Answer": "response_text"})

# Convert the dataset to Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Print to confirm dataset structure
print(dataset)


Dataset({
    features: ['input_text', 'response_text', 'Type'],
    num_rows: 8
})


In [6]:
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    # Tokenize the input and output text
    inputs = tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=512)
    labels = tokenizer(examples['response_text'], truncation=True, padding='max_length', max_length=512)
    inputs['labels'] = labels['input_ids']
    return inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=200,
    weight_decay=0.01,
)


In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.616007
2,No log,0.560660
3,No log,0.518705
4,No log,0.483280
5,No log,0.453344
6,No log,0.425661
7,No log,0.398389
8,No log,0.371651
9,No log,0.346814
10,No log,0.324421


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


TrainOutput(global_step=400, training_loss=0.06924915790557862, metrics={'train_runtime': 89.8527, 'train_samples_per_second': 17.807, 'train_steps_per_second': 4.452, 'total_flos': 81655234560000.0, 'train_loss': 0.06924915790557862, 'epoch': 200.0})

In [28]:
def get_response(input_text, temperature=0.1, max_length=600): # Adjust max_length here
    # You need to define or import the 'chatbot' function here.
    # For example, if you are using a pipeline from transformers:
    from transformers import pipeline
    chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

    response = chatbot(input_text, max_length=max_length, num_return_sequences=1, temperature=temperature)
    return response[0]["generated_text"]



In [29]:
# Save the potentially improved model
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


('/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt/tokenizer.json')

In [31]:
# Check if GPU is available
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("GPU is available and being used.")
else:
  device = torch.device("cpu")
  print("GPU is not available, using CPU.")

# Move model to GPU if available
model.to(device)

GPU is available and being used.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [32]:
# Reload the model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
# Import the pipeline function
from transformers import pipeline

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt')
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Colab Notebooks/Langchain/NewFt')


def get_response(input_text, temperature=0.1, max_length=600):
    # Pass the device to the pipeline
    chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

    response = chatbot(input_text, max_length=max_length, num_return_sequences=1, temperature=temperature)
    return response[0]["generated_text"]

In [1]:
user_input = "What is the head office address of GM Fashions?"
response = get_response(user_input)
print(response)

NameError: name 'get_response' is not defined